In [1]:
from config import api_key
import xml.etree.ElementTree as ET
import requests 
import xmltodict
import json
import pprint
import pandas as pd

In [2]:
path = "../Resources/bestsellers with categories.csv"
df = pd.read_csv(path)
df.columns

Index(['Name', 'Author', 'User Rating', 'Reviews', 'Price', 'Year', 'Genre'], dtype='object')

In [3]:
#sort DataFrame by 'User Rating'
df = df.sort_values("User Rating",ascending=False)

In [4]:
#Drop duplicates
df = df.drop_duplicates('Author')

In [5]:
#convert authors'names into a list
author_list = df['Author'].tolist()

In [6]:
len(author_list)

248

In [7]:
# get Authors' ID

In [8]:
authors_name_and_ids = []
for name in author_list:

    query=name
    url = f"https://www.goodreads.com/api/author_url/{query}?key={api_key}"

    r = requests.get(url)
    root = ET.fromstring(r.text)
    tree = ET.ElementTree(root)
    tree.write("list.xml")

    with open("list.xml") as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()

    json_data = json.dumps(data_dict)
    with open("data.json","w") as json_file:
        json_file.write(json_data)
        
    
    with open("data.json") as f:
        data = json.load(f)
        authors_name_and_ids.append(data)

In [15]:
author_id_list=[]
author_name_list=[]
author_link_list=[]

for info in authors_name_and_ids:
    if "author" in info['GoodreadsResponse']:
        author_id = info['GoodreadsResponse']['author']['@id']
        author_name = info['GoodreadsResponse']['author']['name']
        author_link = info['GoodreadsResponse']['author']['link']

        author_id_list.append(info['GoodreadsResponse']['author']['@id'])
        author_name_list.append(author_name)
        author_link_list.append(author_link)
    else:
        pass

In [10]:
df_author = pd.DataFrame({"author_id":author_id_list,
                          "author_name":author_name_list
                          })

In [11]:
df_author

,author_id,author_name
0,14980615,Chip Gaines
1,33280,Dav Pilkey
2,61105,Dr. Seuss
3,3362,Eric Carle
4,4880403,Emily Winfield Martin
...,...,...
242,19157082,Ian K. Smith M.D.
243,2383,Gillian Flynn
244,3309823,Stephen Gallup
245,643126,Muriel Barbery


In [12]:
df_author.to_csv("files/author_id.csv")

In [13]:
#API pull Authors Information

In [14]:
author_info_list = []
for id in author_id_list:
    query = id
    url=f'https://www.goodreads.com/author/show/{id}?&key={api_key}'
    r = requests.get(url)
    root = ET.fromstring(r.text)
    tree = ET.ElementTree(root)
    tree.write("author_info.xml")

    with open("author_info.xml") as xml_file:
        data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()

    json_data = json.dumps(data_dict)
    with open("author_info.json","w") as json_file:
        json_file.write(json_data)
        

    with open("author_info.json") as f:
        data = json.load(f)

        author_info_list.append(data)

In [16]:
author_id_list =[]
gender_list = []
hometown_list = []
author_book_list = []


for info in author_info_list:
    author_id = info['GoodreadsResponse']['author']['id']
    author_id_list.append(author_id)

    if 'gender' in info['GoodreadsResponse']['author']:
       gender_list.append(info['GoodreadsResponse']['author']['gender'])

    
    if 'hometown' in info['GoodreadsResponse']['author']:
        hometown_list.append(info['GoodreadsResponse']['author']['hometown'])

    
    # each_author_book_dict = []
    books = info['GoodreadsResponse']['author']['books']['book']

    for book in books:
        try:
            book_title = book['title']
            book_isbn = book['isbn']
            book_isbn13 = book['isbn13']
            book_average_rating = book['average_rating']
            book_publisher = book['publisher']
            certain_author_book_dict={'author_id':author_id,
                                      'book_title':book_title,
                                      'book_isbn':book_isbn,
                                      'book_isbn13':book_isbn13,
                                      'book_average_rating':book_average_rating,
                                      'book_publiser':book_publisher}
            author_book_list.append(certain_author_book_dict)
        except TypeError:
            pass

       

In [17]:
duplicated_author_id = []
book_title_list = []
book_isbn_list = []
book_isbn13_list = []
book_average_rating_list = []
book_publisher_list = []

for i in author_book_list:
    duplicated_author_id.append(i['author_id'])
    book_title_list.append(i['book_title'])
    book_isbn_list.append(i['book_isbn'])
    book_isbn13_list.append(i['book_isbn13'])
    book_average_rating_list.append(i['book_average_rating'])
    book_publisher_list.append(i['book_publiser'])

In [18]:
each_author_book_info_dict = pd.DataFrame({'author_id':duplicated_author_id,
                                           'book_title':book_title_list,
                                           'book_isbn':book_isbn_list,
                                           'book_isbn13':book_isbn13_list,
                                           'book_average_rating':book_average_rating_list,
                                           'book_publisher':book_publisher_list
                                            })  

In [19]:
each_author_book_info_dict

,author_id,book_title,book_isbn,book_isbn13,book_average_rating,book_publisher
0,14980615,The Magnolia Story,{'@nil': 'true'},{'@nil': 'true'},4.18,None
1,14980615,Capital Gaines: Smart Things I Learned Doing S...,0785216308,9780785216308,4.01,Thomas Nelson
2,14980615,Building a Network: The Power of Relationships...,1713529122,9781713529125,3.00,Thomas Nelson on Brilliance Audio
3,14980615,Homebody / Magnolia Table,8925599376,9788925599373,4.33,None
4,14980615,What Makes a Marriage Last: 40 Celebrated Coup...,{'@nil': 'true'},{'@nil': 'true'},4.08,HarperAudio
...,...,...,...,...,...,...
2203,8719,The Ambush,{'@nil': 'true'},{'@nil': 'true'},3.80,Tin House Magazine
2204,8719,Het Land van de Papaver,9023416066,9789023416067,3.68,De Bezige Bij
2205,8719,Eine Strumpfbandnatter,3899406990,9783899406993,3.87,Der Hörverl
2206,8719,Tam O'Shanter,{'@nil': 'true'},{'@nil': 'true'},3.61,None


In [20]:
df_author_info = pd.DataFrame({'author_id':author_id_list,
                  'gender':gender_list,
                  'hometown':hometown_list
                  })  
df_author_info.head()

,author_id,gender,hometown
0,14980615,female,"Albuquerque, New Mexico"
1,33280,None,"Cleveland, OH"
2,61105,male,"Springfield, MA"
3,3362,male,"Syracuse, New York"
4,4880403,female,None


In [21]:
df_author_and_books = pd.merge(df_author_info,each_author_book_info_dict,how='left',left_on='author_id',right_on='author_id')
df_author_and_books 

,author_id,gender,hometown,book_title,book_isbn,book_isbn13,book_average_rating,book_publisher
0,14980615,female,"Albuquerque, New Mexico",The Magnolia Story,{'@nil': 'true'},{'@nil': 'true'},4.18,None
1,14980615,female,"Albuquerque, New Mexico",Capital Gaines: Smart Things I Learned Doing S...,0785216308,9780785216308,4.01,Thomas Nelson
2,14980615,female,"Albuquerque, New Mexico",Building a Network: The Power of Relationships...,1713529122,9781713529125,3.00,Thomas Nelson on Brilliance Audio
3,14980615,female,"Albuquerque, New Mexico",Homebody / Magnolia Table,8925599376,9788925599373,4.33,None
4,14980615,female,"Albuquerque, New Mexico",What Makes a Marriage Last: 40 Celebrated Coup...,{'@nil': 'true'},{'@nil': 'true'},4.08,HarperAudio
...,...,...,...,...,...,...,...,...
2268,8719,female,"Greenwood, Mississippi",The Ambush,{'@nil': 'true'},{'@nil': 'true'},3.80,Tin House Magazine
2269,8719,female,"Greenwood, Mississippi",Het Land van de Papaver,9023416066,9789023416067,3.68,De Bezige Bij
2270,8719,female,"Greenwood, Mississippi",Eine Strumpfbandnatter,3899406990,9783899406993,3.87,Der Hörverl
2271,8719,female,"Greenwood, Mississippi",Tam O'Shanter,{'@nil': 'true'},{'@nil': 'true'},3.61,None


In [22]:
df_author_and_books.to_csv("files/author_and_books.csv")